In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from tqdm import tqdm
warnings.filterwarnings("ignore")
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [2]:
import xgboost as xgb
import pandas_profiling
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import LinearSVR

In [3]:
file_train_name='./Benz_data/train.csv'
file_test_name='./Benz_data/test.csv'

In [4]:
Train=pd.read_csv(file_train_name,index_col='ID')
Test=pd.read_csv(file_test_name,index_col='ID')
target=Train['y']
Train.drop(['y'],axis=1,inplace=True)

In [5]:
File=pandas_profiling.ProfileReport(Train)
File.to_file(outputfile="output.html")

In [6]:
print("Train shape",Train.shape)
print("Test shape",Test.shape)

Train shape (4209, 376)
Test shape (4209, 376)


In [7]:
Train.head()

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
ID,,,,,,,,,,,,,,,,,,,,,
0,k,v,at,a,d,u,j,o,0,0,...,0,0,1,0,0,0,0,0,0,0
6,k,t,av,e,d,y,l,o,0,0,...,1,0,0,0,0,0,0,0,0,0
7,az,w,n,c,d,x,j,x,0,0,...,0,0,0,0,0,0,1,0,0,0
9,az,t,n,f,d,x,l,e,0,0,...,0,0,0,0,0,0,0,0,0,0
13,az,v,n,f,d,h,d,n,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
print("feature number %d"%(Train.shape[1]))
print('-----------------------------------')
Train.dtypes.value_counts()

feature number 376
-----------------------------------


int64     368
object      8
dtype: int64

In [9]:
feature=[[],[],[],[]]
for c in Train.columns:
    t=Train[c].dtype
    u=len(np.unique(Train[c]))
    if t==object:
        feature[0].append(c)
    elif u==1:
        feature[1].append(c)
    elif u==2:
        feature[2].append(c)
    else:
        feature[3].append(c)
    
print("String:%d\nConstant:%d\n2 kind:%d\nother kind:%d"%(len(feature[0]),len(feature[1]),len(feature[2]),len(feature[3])))
print("String features:",feature[0])
print("Constant features:",feature[1])

String:8
Constant:12
2 kind:356
other kind:0
String features: ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']
Constant features: ['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347']


In [10]:
Full=Train.append(Test)

In [11]:
tmp=File.get_rejected_variables()
tmp.extend(feature[1])

In [12]:
new_Full=Full.drop(tmp,axis=1)

pre=OneHotEncoder(sparse=False)
pre.fit(new_Full[feature[0]])
tmp=pre.transform(new_Full[feature[0]])
pre_Full=pd.concat([new_Full,pd.DataFrame(tmp,index=new_Full.index)],axis=1)
Last_Full=pre_Full.drop(feature[0],axis=1)

Train,Test=Last_Full[:4209],Last_Full[4209:]

In [28]:
params={
        'booster':'gbtree',
        'learning_rate': 0.1, 
        'n_estimators': 55, 
        'max_depth': 3, 
        'min_child_weight': 12, 
        'subsample': 0.7, 
        'colsample_bytree': 0.99, 
        'gamma': 0.001, 
        'reg_alpha': 0.1, 
        'reg_lambda': 3
}

# cv_params={
#     'reg_lambda':[3,4,5]
# }

reg_xgb=xgb.XGBRegressor(**params)
# reg_gbdt=GradientBoostingRegressor()
# cv=GridSearchCV(estimator=reg_xgb,param_grid=cv_params,scoring='r2',cv=30,verbose=1,n_jobs=-1)
# cv.fit(Train,target)
# print(cv.best_score_)
# print(cv.best_params_)    

In [29]:
reg_xgb.fit(Train,target)
Ans=reg_xgb.predict(Test)
csv=pd.DataFrame({'ID':Test.index,'y':Ans})
csv.to_csv('XXX.csv',index=False)

[17:16:57] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


## --------1.3 end---------

In [154]:
kfold=KFold(n_splits=100)
tmp_train,tmp_test=[[],[]],[[],[]]
for u,v in kfold.split(Train):
    reg_xgb.fit(Train.iloc[u,:],target.iloc[u])
    tmp_train[0].extend(reg_xgb.predict(Train.iloc[v,:]))
    tmp_test[0].append(reg_xgb.predict(Test))

for u,v in kfold.split(Train):
    reg_gbdt.fit(Train.iloc[u,:],target.iloc[u])
    tmp_train[1].extend(reg_gbdt.predict(Train.iloc[v,:]))
    tmp_test[1].append(reg_gbdt.predict(Test))

[18:54:44] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:54:50] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:54:56] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:55:02] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:55:08] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:55:15] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:55:21] WARNI

[19:04:40] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:04:46] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:04:52] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:04:58] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:05:04] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:05:10] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:05:16] WARNI

[19:14:36] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:14:42] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:14:48] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:14:54] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [155]:
new_train=np.array([tmp_train[0],tmp_train[1]])
new_test=np.array([np.mean(tmp_test[0],axis=0),np.mean(tmp_test[1],axis=0)])
reg_xgb.fit(new_train.T,target)
Ans=reg_xgb.predict(new_test.T)

In [196]:
csv=pd.DataFrame({'ID':Test.index,'y':Ans})
csv.to_csv('XXX.csv',index=False)

[13:25:38] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
